In [1]:
from stop_words import get_stop_words
import re

In [36]:
'''# create spark session
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark Count words") \
    .getOrCreate()

'''

In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Python Spark Count words").setMaster("local[2]")
sc = SparkContext(conf=conf)   

# https://stackoverflow.com/questions/39882218/py4jjavaerror-an-error-occurred-while-calling-zorg-apache-spark-api-python-pyt

In [32]:
# Testdata
file = sc.wholeTextFiles("data/test/*/*.txt")
filesByLang = file.map(lambda x: (x[0].split("/")[-2], x[1])).reduceByKey(lambda v1,v2: v1+ " " + v2)
wordsByLang = filesByLang.flatMapValues(lambda l: re.findall("\w+", l))

preprocessedWords = wordsByLang.map(lambda x: (x[0], x[1].lower()))\
        .filter(lambda x: (x[1] not in get_stop_words(x[0].lower())))

occurences = preprocessedWords.map(lambda w: ((w[0], w[1]), 1))\
        .reduceByKey(lambda v1,v2: v1+v2)
occurencesPerLanguage = occurences.sortBy(lambda a: a[1], ascending=False)\
        .filter(lambda x: len(x[0][1]) != 1)\
        .map(lambda x: (x[0][0], (x[0][1], x[1])))\
        .groupByKey()



occurencesPerLanguageCollected = occurencesPerLanguage.mapValues(list).collect()

In [35]:
for languages in occurencesPerLanguageCollected:
    print(languages[0] + str(languages[1][:10]))

German[('test', 3), ('hallo', 2), ('neuer', 1), ('tolle', 1)]
English[('hey', 3), ('test', 1), ('never', 1), ('know', 1), ('say', 1), ('easy', 1)]


In [37]:
# real data
file = sc.wholeTextFiles("data/texts/*/*.txt")
filesByLang = file.map(lambda x: (x[0].split("/")[-2], x[1])).reduceByKey(lambda v1,v2: v1+ " " + v2)
wordsByLang = filesByLang.flatMapValues(lambda l: re.findall("\w+", l))

preprocessedWords = wordsByLang.map(lambda x: (x[0], x[1].lower()))\
        .filter(lambda x: (x[1] not in get_stop_words(x[0].lower())))

occurences = preprocessedWords.map(lambda w: ((w[0], w[1]), 1))\
        .reduceByKey(lambda v1,v2: v1+v2)
occurencesPerLanguage = occurences.sortBy(lambda a: a[1], ascending=False)\
        .filter(lambda x: len(x[0][1]) != 1)\
        .map(lambda x: (x[0][0], (x[0][1], x[1])))\
        .groupByKey()



occurencesPerLanguageCollected = occurencesPerLanguage.mapValues(list).collect()

In [41]:
print("Top 10 per language")
for languages in occurencesPerLanguageCollected:
    print(languages[0] + "--> " + str(languages[1][:10]) + "\n")

Top 10 per language
English--> [('said', 18368), ('one', 17930), ('will', 12828), ('upon', 12763), ('now', 11810), ('man', 11763), ('time', 9585), ('well', 9482), ('can', 8902), ('mr', 8743)]

German--> [('sagte', 10700), ('ja', 8849), ('schon', 7948), ('mehr', 7026), ('wurde', 6489), ('ganz', 6165), ('sah', 5858), ('rief', 5683), ('wohl', 5647), ('hand', 5610)]

Russian--> [('ним', 460), ('глаза', 298), ('друг', 273), ('сердце', 271), ('моей', 255), ('ночь', 241), ('жизни', 234), ('свой', 231), ('знаю', 224), ('горский', 224)]

French--> [('plus', 26737), ('dit', 22433), ('bien', 18336), ('homme', 10266), ('dont', 7398), ('rien', 7322), ('puis', 7193), ('monsieur', 7084), ('après', 6781), ('point', 6670)]

Italian--> [('disse', 8759), ('egli', 8691), ('poi', 7288), ('quel', 7199), ('quando', 6762), ('due', 6461), ('così', 6239), ('cosa', 5989), ('ora', 5809), ('ella', 5401)]

Spanish--> [('si', 5444), ('don', 3073), ('tan', 3026), ('dijo', 2992), ('usted', 2985), ('casa', 2451), ('bie